# Inert Dilution

This notebook describes the method by which partial fill pressures are calculated for the ONR detonation project.

## Imports and global definitions
A janky-looking relative import is used for functions within this package, which is still under development. Pint is used to prevent unit-based idiocy on the part of the researcher.

In [1]:
import numpy as np
import cantera as ct
import pandas as pd
import pint
import os
import sys
import itertools
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from funcs import specific_heat_matching as sp
ureg = pint.UnitRegistry()
quant = ureg.Quantity

## Calculation of partial pressures
The partial pressure of the $i^{th}$ component in a mixture, $p_{i}$, is defined as 

<center>$p_{i} = x_{i} P \qquad$ (1)</center>

where $P$ is the mixture total pressure. Since 

<center>$\sum_{i}^{N} x_{i} = 1 \qquad$ (2)</center>

and I am filling with $N=3$ distinct components (fuel, oxidizer, diluent),

<center>$x_{oxidizer} = 1 - (x_{fuel} + x_{diluent}) \qquad$ (3),</center>

which is important because, in the case of $N_{2}$ dilution where air is used as an oxidizer, $N_{2}$ is added in both the oxidizer and diluent streams. Assuming that the fuel stream does not contain the diluent, equations 1-3 can reliably used to calculate fill pressures for each component. It is important to note here that **when the diluent is also a component of the oxidizer** $x_{diluent}$ does *not* represent the total mole fraction of the diluent species in the mixture, but rather the *additional* mole fraction needed in order to match the adiabatic flame temperature achieved with the active diluent.

In [2]:
def calculate_partial_pressures(
    x_diluent:float, 
    x_fuel:float, 
    total_pressure:float
):
    check_array = np.array([x_fuel, x_diluent])
    if any(check_array > 1.) or any(check_array < 0.):
        raise ValueError("Please keep all mole fractions on 0 < x < 1")
    elif sum(check_array) > 1.:
        raise ValueError("Mole fractions cannot sum to > 1")
        
    return {
        "diluent": x_diluent * total_pressure, 
        "fuel": x_fuel * total_pressure, 
        "oxidizer": (1 - (x_diluent + x_fuel)) * total_pressure
    }

## Calculate mixture-specific fill pressures
Fuel and diluent mole fractions are calculated for an inert-diluted mixture using adiabatic flame temperature matching and dilution functions. Calculated mole fractions are then fed in to `calculate_partial_pressures()` to generate the fill cutoff pressures.

In [3]:
def calculate_fill_pressures(
    mech,
    fuel,
    oxidizer,
    phi,
    diluent_active,
    mol_frac,
    diluent_inert,
    initial_temp,
    initial_press,
    return_inert_mol_frac=True
):
    mol_frac_inert = sp.match_adiabatic_temp(
        mech,
        fuel,
        oxidizer,
        phi,
        diluent_active,
        mol_frac,
        diluent_inert,
        initial_temp,
        initial_press
    )

    gas = ct.Solution(mech)
    gas.set_equivalence_ratio(phi, fuel, oxidizer)
    mol_frac_fuel = sp.diluted_species_dict(
        gas.mole_fraction_dict(),
        diluent_inert,
        mol_frac_inert
    )[fuel]
    
    if return_inert_mol_frac:
        return calculate_partial_pressures(
            x_diluent=mol_frac_inert,
            x_fuel=mol_frac_fuel,
            total_pressure=initial_press
        ), mol_frac_inert
        
    else:
        return calculate_partial_pressures(
            x_diluent=mol_frac_inert,
            x_fuel=mol_frac_fuel,
            total_pressure=initial_press
        )

## Generate inert-diluted test matrix

In [5]:
# csv output
output_path = os.path.join(
    "D:\\",
    "inert test matrix.csv"
)

# iterable quantities
fuels = ["C3H8"]
oxidizers = {"air": "O2:1 N2:3.76"}
active_mol_fracs = np.array([0, 5.5, 10.]) / 100
inert_diluents = ["N2"]
init_pressures = [ct.one_atm]
equivs = [0.4, 0.7, 1.0]

# constant quantities
diluent_active = "CO2"
initial_temp = quant(60, "degF").to("K").magnitude
mech = "gri30.cti"

output_columns = [
    "fuel",
    "oxidizer",
    "diluent",
    "diluent_mol_frac",
    "equivalence",
    "init_pressure",
    "p_dil",
    "p_ox",
    "p_f"
]

combinations = list(itertools.product(
    fuels,
    oxidizers,
    inert_diluents,
    equivs,
    active_mol_fracs,
    init_pressures
))

data = np.empty((len(output_columns), len(combinations)), dtype=object)
for i, (f, ox, dil, phi, mf_active, p_init) in enumerate(combinations): 
    p_fill, mf_inert = calculate_fill_pressures(
        mech,
        f,
        oxidizers[ox],
        phi,
        diluent_active,
        mf_active,
        dil,
        initial_temp,
        p_init,
        return_inert_mol_frac=True
    )
    data[i, :] = (
        f, 
        ox,
        dil,
        mf_inert,
        phi,
        p_init / ct.one_atm,
        p_fill["diluent"],
        p_fill["oxidizer"],
        p_fill["fuel"]
    )
df_out = pd.DataFrame(data=data, columns=output_columns)
df_out.to_csv(output_path, index=False)